In [ ]:
%pylab inline

# My implementation of method-of-moments

# The program reads "ps2_ibd.lwk.chr1.ped", which contains chromosome 1 SNPs data
# for 97 individuals, and estimates % IBD for each pair of individuals.

# Outputs the result to "my.ibd.genome"

import pandas as pd
from itertools import combinations

def main():
    df = pd.read_csv("ps2_ibd.lwk.chr1.ped", sep=" ", header=None)
    samples_to_alleles = {}
    
    for _, row in df.iterrows():
        samples_to_alleles[row[0]] = list(row[6:])
        
    fout = open("my.ibd.genome", 'w')
    fout.write("ID1 ID2 Z0 Z1 Z2 PI\n") # header
    
    for id1, id2 in combinations(samples_to_alleles.keys(), 2): # for each pair
        compute_Z(id1, id2, samples_to_alleles, fout)

    fout.close()
    

def compute_Z(id1, id2, samples_to_alleles, fout):    
    alleles1 = samples_to_alleles[id1]
    alleles2 = samples_to_alleles[id2]
    
    N_IBS0, N_IBS1, N_IBS2 = 0, 0, 0
       
    N_I0Z0, N_I1Z0, N_I1Z1, N_I2Z0, N_I2Z1 = 0, 0, 0, 0, 0
    N_I2Z2 = len(alleles1) / 2
    
    T_A = len(samples_to_alleles) * 2
    T_A123 = (T_A / (T_A - 1)) * (T_A / (T_A - 2)) * (T_A / (T_A - 3))
    T_A12 = (T_A / (T_A - 1)) * (T_A / (T_A - 2))
            
    for i in range(0, len(alleles1), 2): # for each SNP
        X = 0 # allele 1 count
        Y = 0 # allele 2 count
        
        genotype1 = alleles1[i] + alleles1[i+1]
        genotype2 = alleles2[i] + alleles2[i+1]
        
        if abs(genotype1 - genotype2) == 2:
            N_IBS0 += 1
        elif abs(genotype1 - genotype2) == 1:
            N_IBS1 += 1
        else:
            N_IBS2 += 1
        
        for alleles in samples_to_alleles.values():
            for allele in (alleles[i], alleles[i+1]): 
                if allele == 1:
                    X += 1
                else:
                    Y += 1
                
        p = X / T_A
        q = Y / T_A
        
        if p != 0 and q != 0:
            N_I0Z0 += 2 * pow(p, 2) * pow(q, 2) * ((X - 1) / X) * ((Y - 1) / Y) * T_A123

            N_I1Z0 += 4 * pow(p, 3) * q * ((X - 1) / X) * ((X - 2) / X) * T_A123 + \
                      4 * p * pow(q, 3) * ((Y - 1) / Y) * ((Y - 2) / Y) * T_A123

            N_I1Z1 += 2 * pow(p, 2) * q * ((X - 1) / X) * T_A12 + \
                      2 * p * pow(q, 2) * ((Y - 1) / Y) * T_A12
        
        if p == 0 and q != 0: 
            N_I2Z0 += pow(q, 4) * ((Y - 1) / Y) * ((Y - 2) / Y) * ((Y - 3) / Y) * T_A123
            
            N_I2Z1 += pow(q, 3) * ((Y - 1) / Y) * ((Y - 2) / Y) * T_A12
        elif q == 0 and p != 0:
            N_I2Z0 += pow(p, 4) * ((X - 1) / X) * ((X - 2) / X) * ((X - 3) / X) * T_A123
            
            N_I2Z1 += pow(p, 3) * ((X - 1) / X) * ((X - 2) / X) * T_A12  
        elif p != 0 and q != 0:
            N_I2Z0 += pow(p, 4) * ((X - 1) / X) * ((X - 2) / X) * ((X - 3) / X) * T_A123 + \
                      pow(q, 4) * ((Y - 1) / Y) * ((Y - 2) / Y) * ((Y - 3) / Y) * T_A123 + \
                      4 * pow(p, 2) * pow(q, 2) * ((X - 1) / X) * ((Y - 1) / Y) * T_A123

            N_I2Z1 += pow(p, 3) * ((X - 1) / X) * ((X - 2) / X) * T_A12 + \
                      pow(q, 3) * ((Y - 1) / Y) * ((Y - 2) / Y) * T_A12 + \
                      pow(p, 2) * q * ((X - 1) / X) * T_A12 + \
                      p * pow(q, 2) * ((Y - 1) / Y) * T_A12
        
    Z0 = N_IBS0 / N_I0Z0
    Z1 = (N_IBS1 - Z0 * N_I1Z0) / N_I1Z1
    Z2 = (N_IBS2 - Z0 * N_I2Z0 - Z1 * N_I2Z1) / N_I2Z2
    pi = Z1 / 2 + Z2
    
    Z0, Z1, Z2, pi = bound_Z(Z0, Z1, Z2, pi)
    
    fout.write(f"{id1} {id2} {Z0} {Z1} {Z2} {pi}\n")

def bound_Z(Z0, Z1, Z2, pi):
    if Z0 > 1:
        Z0, Z1, Z2 = 1, 0, 0
    elif Z1 > 1:
        Z0, Z1, Z2 = 0, 1, 0
    elif Z2 > 1:
        Z0, Z1, Z2 = 0, 0, 1   
    elif Z0 < 0:
        S = Z1 + Z2
        Z0, Z1, Z2 = 0, Z1 / S, Z2 / S
    elif Z1 < 0:
        S = Z0 + Z2
        Z0, Z1, Z2 = Z0 / S, 0, Z2 / S
    elif Z2 < 0:
        S = Z0 + Z1
        Z0, Z1, Z2 = Z0 / S, Z1 / S, 0    
    elif pow(pi, 2) <= Z2:
        Z0 = (1 - pi) ** 2
        Z1 = 2 * pi * (1 - pi)
        Z2 = pi ** 2

    return Z0, Z1, Z2, Z1 / 2 + Z2
    

if __name__ == '__main__':
    main()

In [ ]:
%%bash

# Run PLINK's method-of-moments on the same dataset
# but with all autosomal SNPs

# Outputs the result to "plink.ibd.genome"

# DATADIR=/datasets/cs284-sp21-A00-public/ps2
# plink --genome --bfile ${DATADIR}/ps2_ibd.lwk --out plink.ibd

In [ ]:
%pylab inline

# Plot IBD=0 vs IBD=1 for my result and PLINK's result

import matplotlib.pyplot as plt

true_ibd = open("plink.ibd.genome") # PLINK's result
next(true_ibd)  # skip header

IBD0, IBD1 = [], []

for line in true_ibd.readlines():
    row = line.split()
    IBD0.append(float(row[6]))
    IBD1.append(float(row[7]))

plt.figure()
plt.scatter(IBD0, IBD1, s=15)
plt.title("PLINK's Implementation")
plt.xlabel("IBD0")
plt.ylabel("IBD1")
plt.show()

ibd = open("my.ibd.genome") # my result
next(ibd)  # skip header

IBD0, IBD1 = [], []

for line in ibd.readlines():
    row = line.split()
    IBD0.append(float(row[2]))
    IBD1.append(float(row[3]))

plt.figure()
plt.scatter(IBD0, IBD1, s=15)
plt.title("My Implementation")
plt.xlabel("IBD0")
plt.ylabel("IBD1")
plt.show()

In [ ]:
%pylab inline

# Calculate MSE for pi, which is P(IBD = 2) + 0.5 * P(IBD = 1),
# between my result and PLINK's result

import numpy as np

plink_ibd = open("plink.ibd.genome")
my_ibd = open("my.ibd.genome")
next(plink_ibd)
next(my_ibd)

plink_pi, my_pi = [], []

for line in plink_ibd.readlines():
    row = line.split()
    plink_pi.append(float(row[9]))

for line in my_ibd.readlines():
    row = line.split()
    my_pi.append(float(row[5]))

MSE = sum(pow((np.array(plink_pi) - np.array(my_pi)), 2)) / len(plink_pi)
print(MSE)